In [4]:
import time
from flask import Flask, request, jsonify
from collections import namedtuple
import pandas as pd
import json
from pytrends.request import TrendReq
import requests
from statistics import mean
import sys
from tqdm import tqdm
import sys
import geopy.distance


### nxn+1 distance+popularity matrix


In [6]:
pytrend = TrendReq()
startTime = time.time()

In [22]:
cities = pd.read_csv('cities_20k.csv')

In [8]:
cities.head()

,Unnamed: 0,Woonplaatsen,Provincie,Landsdeel,Gemeente,Latitude,Longitude,Population,Coordinates
0,12,Assen,Drenthe,Noord-Nederland,Assen,52.983333,6.550000,68798,"52° 59′ NB, 6° 33′ OL"
1,29,Coevorden,Drenthe,Noord-Nederland,Coevorden,52.650000,6.733333,35303,"52° 39′ NB, 6° 44′ OL"
2,64,Emmen,Drenthe,Noord-Nederland,Emmen,52.783333,6.900000,56545,"52° 47′ NB, 6° 54′ OL"
3,98,Hoogeveen,Drenthe,Noord-Nederland,Hoogeveen,52.716667,6.466667,40190,"52° 43′ NB, 6° 28′ OL"
4,115,Meppel,Drenthe,Noord-Nederland,Meppel,52.700000,6.183333,34125,"52° 42′ NB, 6° 11′ OL"


In [9]:
city_names = cities['Woonplaatsen'].unique()
len_city_names = len(city_names)
popularity = [0] * len_city_names

In [10]:
new_df = pd.DataFrame(list(zip(city_names, popularity)), columns = ['City', 'Popularity']) 

In [11]:
new_df.head()

,City,Popularity
0,Assen,0
1,Coevorden,0
2,Emmen,0
3,Hoogeveen,0
4,Meppel,0


In [12]:
new_df['City'] = city_names

In [13]:
new_df['Popularity'] = popularity

In [14]:
new_df.head()

,City,Popularity
0,Assen,0
1,Coevorden,0
2,Emmen,0
3,Hoogeveen,0
4,Meppel,0


In [15]:
for index, row in new_df.iterrows():
    # print(row['c1'], row['c2'])
    popularity = pytrend.build_payload([row['City']], cat=0, timeframe='today 5-y', geo='', gprop='')
    interest_df = pytrend.interest_over_time()
    interest = mean(interest_df[row['City']])
    print(interest)
    new_df.loc[index, 'Popularity'] = interest

20.392307692307693
49.78846153846154
60.12307692307692
76.37307692307692
49.45384615384615
73.45384615384616
73.08461538461539
74.97692307692307
65.5576923076923
18.1
17.546153846153846
62.59615384615385
43.57692307692308
46.63846153846154
67.75769230769231
76.48076923076923
77.82307692307693
25.023076923076925
54.52307692307692
61.15384615384615
73.72307692307692
43.10769230769231
79.66538461538461
68.47692307692307
63.83461538461538
71.68846153846154
74.3076923076923
62.90384615384615
47.85
65.42692307692307
74.17307692307692
49.22692307692308
71.01153846153846
65.31153846153846
68.42692307692307
71.28846153846153
80.28076923076924
49.94615384615385
74.29230769230769
66.26153846153846
48.91153846153846
64.26923076923077
66.03076923076924
62.026923076923076
72.69615384615385
46.63076923076923
70.84615384615384
58.58846153846154
61.68846153846154
72.48846153846154
75
73.45769230769231
73.73076923076923
80.09615384615384
47.11153846153846
77.31923076923077
80.32307692307693
44.003846153

In [16]:
new_df.head()

,City,Popularity
0,Assen,20.392308
1,Coevorden,49.788462
2,Emmen,60.123077
3,Hoogeveen,76.373077
4,Meppel,49.453846


In [27]:
all_jobs = len_city_names * len_city_names
count = 1
toolbar_width = 40
for col in city_names:
    column = []
    start_time = time.time()
    for row in city_names:     
        c1_la = float(cities.loc[cities['Woonplaatsen'] == col, 'Latitude'])
        c1_lo = float(cities.loc[cities['Woonplaatsen'] == col, 'Longitude'])
        c2_la = float(cities.loc[cities['Woonplaatsen'] == row, 'Latitude'])
        c2_lo = float(cities.loc[cities['Woonplaatsen'] == row, 'Longitude'])
        coords_1 = (c1_la, c1_lo)
        #print(float(c1_la))
        #print(c1_lo)
        coords_2 = (c2_la, c2_lo)
        #print(coords_2)
        distance = geopy.distance.distance(coords_1, coords_2).km
        column.append(distance)
        division = count/all_jobs
        #print (str(count) + '/' + str(all_jobs))
        count+=1
        #print(column)
    end_time = time.time()
    print("total time taken this loop: ", end_time - start_time)
    new_df[col] = column
    print(new_df.head())

total time taken this loop:  0.7759318351745605
        City  Popularity      Assen
0      Assen   20.392308   0.000000
1  Coevorden   49.788462  39.099434
2      Emmen   60.123077  32.411371
3  Hoogeveen   76.373077  30.201962
4     Meppel   49.453846  40.057228
total time taken this loop:  0.8347673416137695
        City  Popularity      Assen  Coevorden
0      Assen   20.392308   0.000000  39.099434
1  Coevorden   49.788462  39.099434   0.000000
2      Emmen   60.123077  32.411371  18.627856
3  Hoogeveen   76.373077  30.201962  19.500270
4     Meppel   49.453846  40.057228  37.615833
total time taken this loop:  0.6432816982269287
        City  Popularity      Assen  Coevorden      Emmen
0      Assen   20.392308   0.000000  39.099434  32.411371
1  Coevorden   49.788462  39.099434   0.000000  18.627856
2      Emmen   60.123077  32.411371  18.627856   0.000000
3  Hoogeveen   76.373077  30.201962  19.500270  30.186348
4     Meppel   49.453846  40.057228  37.615833  49.281842
total time

4   60.097674  46.854104   32.526661  
total time taken this loop:  0.6373002529144287
        City  Popularity      Assen  Coevorden      Emmen  Hoogeveen  \
0      Assen   20.392308   0.000000  39.099434  32.411371  30.201962   
1  Coevorden   49.788462  39.099434   0.000000  18.627856  19.500270   
2      Emmen   60.123077  32.411371  18.627856   0.000000  30.186348   
3  Hoogeveen   76.373077  30.201962  19.500270  30.186348   0.000000   
4     Meppel   49.453846  40.057228  37.615833  49.281842  19.239744   

      Meppel      Almere    Dronten  Emmeloord    Lelystad        Urk  \
0  40.057228  113.322839  76.570713  61.525022   90.806432  74.015415   
1  37.615833  107.693697  70.489979  66.912272   87.479149  76.701937   
2  49.281842  123.176478  85.406324  78.005767  102.017056  89.091572   
3  19.239744   93.321382  55.458929  48.429196   71.905112  59.077692   
4   0.000000   75.356933  37.624593  29.347091   53.415467  39.847084   

     Zeewolde   Drachten  Heerenveen  Lee

[5 rows x 21 columns]
total time taken this loop:  0.7011246681213379
        City  Popularity      Assen  Coevorden      Emmen  Hoogeveen  \
0      Assen   20.392308   0.000000  39.099434  32.411371  30.201962   
1  Coevorden   49.788462  39.099434   0.000000  18.627856  19.500270   
2      Emmen   60.123077  32.411371  18.627856   0.000000  30.186348   
3  Hoogeveen   76.373077  30.201962  19.500270  30.186348   0.000000   
4     Meppel   49.453846  40.057228  37.615833  49.281842  19.239744   

      Meppel      Almere    Dronten  Emmeloord  ...    Zeewolde   Drachten  \
0  40.057228  113.322839  76.570713  61.525022  ...   99.820309  34.634007   
1  37.615833  107.693697  70.489979  66.912272  ...   88.797165  67.908500   
2  49.281842  123.176478  85.406324  78.005767  ...  105.347081  66.250275   
3  19.239744   93.321382  55.458929  48.429196  ...   76.369140  51.442755   
4   0.000000   75.356933  37.624593  29.347091  ...   60.097674  46.854104   

   Heerenveen  Leeuwarden   

[5 rows x 27 columns]
total time taken this loop:  0.8003802299499512
        City  Popularity      Assen  Coevorden      Emmen  Hoogeveen  \
0      Assen   20.392308   0.000000  39.099434  32.411371  30.201962   
1  Coevorden   49.788462  39.099434   0.000000  18.627856  19.500270   
2      Emmen   60.123077  32.411371  18.627856   0.000000  30.186348   
3  Hoogeveen   76.373077  30.201962  19.500270  30.186348   0.000000   
4     Meppel   49.453846  40.057228  37.615833  49.281842  19.239744   

      Meppel      Almere    Dronten  Emmeloord  ...      Arnhem   Barneveld  \
0  40.057228  113.322839  76.570713  61.525022  ...  119.303841  115.082910   
1  37.615833  107.693697  70.489979  66.912272  ...   92.758743   97.129029   
2  49.281842  123.176478  85.406324  78.005767  ...  111.386582  115.069492   
3  19.239744   93.321382  55.458929  48.429196  ...   89.795100   88.452783   
4   0.000000   75.356933  37.624593  29.347091  ...   81.790865   75.116527   

   Beuningen    Culemb

[5 rows x 33 columns]
total time taken this loop:  0.7410171031951904
        City  Popularity      Assen  Coevorden      Emmen  Hoogeveen  \
0      Assen   20.392308   0.000000  39.099434  32.411371  30.201962   
1  Coevorden   49.788462  39.099434   0.000000  18.627856  19.500270   
2      Emmen   60.123077  32.411371  18.627856   0.000000  30.186348   
3  Hoogeveen   76.373077  30.201962  19.500270  30.186348   0.000000   
4     Meppel   49.453846  40.057228  37.615833  49.281842  19.239744   

      Meppel      Almere    Dronten  Emmeloord  ...         Ede        Elst  \
0  40.057228  113.322839  76.570713  61.525022  ...  119.924373  128.304093   
1  37.615833  107.693697  70.489979  66.912272  ...   98.691642  102.129929   
2  49.281842  123.176478  85.406324  78.005767  ...  117.045738  120.757697   
3  19.239744   93.321382  55.458929  48.429196  ...   92.033050   98.941285   
4   0.000000   75.356933  37.624593  29.347091  ...   80.432539   90.373517   

       Ermelo  Harderw

...

[5 rows x 163 columns]
total time taken this loop:  0.8667106628417969
        City  Popularity      Assen  Coevorden      Emmen  Hoogeveen  \
0      Assen   20.392308   0.000000  39.099434  32.411371  30.201962   
1  Coevorden   49.788462  39.099434   0.000000  18.627856  19.500270   
2      Emmen   60.123077  32.411371  18.627856   0.000000  30.186348   
3  Hoogeveen   76.373077  30.201962  19.500270  30.186348   0.000000   
4     Meppel   49.453846  40.057228  37.615833  49.281842  19.239744   

      Meppel      Almere    Dronten  Emmeloord  ...  Hendrik-Ido-Ambacht  \
0  40.057228  113.322839  76.570713  61.525022  ...           181.399856   
1  37.615833  107.693697  70.489979  66.912272  ...           168.788976   
2  49.281842  123.176478  85.406324  78.005767  ...           186.205351   
3  19.239744   93.321382  55.458929  48.429196  ...           157.957778   
4   0.000000   75.356933  37.624593  29.347091  ...           141.905322   

     Hillegom  Hoogvliet Rotterdam     

[5 rows x 168 columns]
total time taken this loop:  1.1738369464874268
        City  Popularity      Assen  Coevorden      Emmen  Hoogeveen  \
0      Assen   20.392308   0.000000  39.099434  32.411371  30.201962   
1  Coevorden   49.788462  39.099434   0.000000  18.627856  19.500270   
2      Emmen   60.123077  32.411371  18.627856   0.000000  30.186348   
3  Hoogeveen   76.373077  30.201962  19.500270  30.186348   0.000000   
4     Meppel   49.453846  40.057228  37.615833  49.281842  19.239744   

      Meppel      Almere    Dronten  Emmeloord  ...      Leiden  Leiderdorp  \
0  40.057228  113.322839  76.570713  61.525022  ...  168.028718  165.212435   
1  37.615833  107.693697  70.489979  66.912272  ...  162.929316  159.735345   
2  49.281842  123.176478  85.406324  78.005767  ...  178.710563  175.593503   
3  19.239744   93.321382  55.458929  48.429196  ...  148.895799  145.822624   
4   0.000000   75.356933  37.624593  29.347091  ...  130.835229  127.839182   

   Leidschendam      

[5 rows x 173 columns]
total time taken this loop:  1.0109784603118896
        City  Popularity      Assen  Coevorden      Emmen  Hoogeveen  \
0      Assen   20.392308   0.000000  39.099434  32.411371  30.201962   
1  Coevorden   49.788462  39.099434   0.000000  18.627856  19.500270   
2      Emmen   60.123077  32.411371  18.627856   0.000000  30.186348   
3  Hoogeveen   76.373077  30.201962  19.500270  30.186348   0.000000   
4     Meppel   49.453846  40.057228  37.615833  49.281842  19.239744   

      Meppel      Almere    Dronten  Emmeloord  ...   Naaldwijk  \
0  40.057228  113.322839  76.570713  61.525022  ...  192.575759   
1  37.615833  107.693697  70.489979  66.912272  ...  186.193235   
2  49.281842  123.176478  85.406324  78.005767  ...  202.386656   
3  19.239744   93.321382  55.458929  48.429196  ...  172.780708   
4   0.000000   75.356933  37.624593  29.347091  ...  154.995443   

   Nieuwerkerk aan den IJssel   Nieuwkoop   Noordwijk  Oegstgeest  \
0                  173.3

[5 rows x 178 columns]
total time taken this loop:  1.0089333057403564
        City  Popularity      Assen  Coevorden      Emmen  Hoogeveen  \
0      Assen   20.392308   0.000000  39.099434  32.411371  30.201962   
1  Coevorden   49.788462  39.099434   0.000000  18.627856  19.500270   
2      Emmen   60.123077  32.411371  18.627856   0.000000  30.186348   
3  Hoogeveen   76.373077  30.201962  19.500270  30.186348   0.000000   
4     Meppel   49.453846  40.057228  37.615833  49.281842  19.239744   

      Meppel      Almere    Dronten  Emmeloord  ...  Oud-Beijerland  \
0  40.057228  113.322839  76.570713  61.525022  ...      195.607222   
1  37.615833  107.693697  70.489979  66.912272  ...      184.407703   
2  49.281842  123.176478  85.406324  78.005767  ...      201.614799   
3  19.239744   93.321382  55.458929  48.429196  ...      173.007312   
4   0.000000   75.356933  37.624593  29.347091  ...      156.481069   

   Papendrecht   Pijnacker  Ridderkerk    Rijswijk   Rotterdam    Sch

[5 rows x 183 columns]
total time taken this loop:  1.2516634464263916
        City  Popularity      Assen  Coevorden      Emmen  Hoogeveen  \
0      Assen   20.392308   0.000000  39.099434  32.411371  30.201962   
1  Coevorden   49.788462  39.099434   0.000000  18.627856  19.500270   
2      Emmen   60.123077  32.411371  18.627856   0.000000  30.186348   
3  Hoogeveen   76.373077  30.201962  19.500270  30.186348   0.000000   
4     Meppel   49.453846  40.057228  37.615833  49.281842  19.239744   

      Meppel      Almere    Dronten  Emmeloord  ...   Rotterdam    Schiedam  \
0  40.057228  113.322839  76.570713  61.525022  ...  183.053775  188.280168   
1  37.615833  107.693697  70.489979  66.912272  ...  172.869376  178.913599   
2  49.281842  123.176478  85.406324  78.005767  ...  189.840850  195.739687   
3  19.239744   93.321382  55.458929  48.429196  ...  160.957728  166.680963   
4   0.000000   75.356933  37.624593  29.347091  ...  144.144147  149.632562   

   Sliedrecht  Spijke

In [29]:
new_df.to_csv('city_matrix.csv')